In [1]:
from matplotlib import pyplot as plt

In [ ]:
m_to_plot = ['waiter',  'landlord',   'salesman','policeman',
                   'monk',   'handyman' , 'congressman', 'headmaster']
n_to_plot = ['socialite', 'homemaker', 'housekeeper', 'nurse', 'nanny',
                     'hooker', 'marshal', 'archbishop', 'captain', 'colonel']

In [ ]:
def show_bias(E, gender_direction, ll, \
              m_to_plot = m_to_plot, n_to_plot = n_to_plot, \
              female_stereotyped_prof = female_stereotyped_prof, male_stereotyped_prof = male_stereotyped_prof, neutral_prof = neutral_prof):
    female_stereotyped_gender = {x: np.dot(E.v(x), gender_direction) \
                                 for x in female_stereotyped_prof if x in E.words}
    male_stereotyped_gender = {x: np.dot(E.v(x), gender_direction) \
                               for x in male_stereotyped_prof if x in E.words}
    neutral_gender = {x: np.dot(E.v(x), gender_direction)  \
                      for x in neutral_prof if x in E.words}
    female_p = list(female_stereotyped_gender.keys())
    male_p = list(male_stereotyped_gender.keys())
    neutral_p = list(neutral_gender.keys())
    y_female = {female_p[x]:x/(len(female_p)) for x in range(len(female_p))}
    y_male = {male_p[x]:x/(len(male_p)) for x in range(len(male_p))}
    y_neutral = {neutral_p[x]:x/(len(neutral_p)) for x in range(len(neutral_p))}
    fig = plt.figure()
    fig.set_size_inches(8, 6)
    f, = plt.plot(list(female_stereotyped_gender.values()), list(y_female.values()), 'ro', label = 'Female')
    m, = plt.plot(list(male_stereotyped_gender.values()), list(y_male.values()), 'ko', label = 'Male')
    n, = plt.plot(list(neutral_gender.values()), list(y_neutral.values()), 'bx', label = 'Neutral')
    
    plotx = [neutral_gender[x] for x in n_to_plot if x in neutral_gender]
    ploty = [y_neutral[x] for x in n_to_plot if x in neutral_gender] 
    plt.plot(plotx, ploty, 'gx', markersize=10)

    plt.yticks([])
    # plt.legend(handles=[f,m,n], loc='best', fontsize = 15)

    for x in female_stereotyped_gender:
        x_l = female_stereotyped_gender[x] 
        y_l = y_female[x]
        plt.annotate(x, (x_l, y_l ), fontsize=15)

    for x in m_to_plot[:]:
        ann_x = male_stereotyped_gender[x]
        plt.annotate(x, (ann_x, y_male[x]), fontsize=15)
    
    for x in n_to_plot:
        if x not in neutral_gender:
            continue
        x_l = neutral_gender[x]
        y_l = y_neutral[x] 
        plt.annotate(x, (x_l, y_l ), fontsize = 15)
    plt.xticks(fontsize=17)
    plt.title(f"Bias analysis in {ll}")
    plt.show()
    plt.close()

In [12]:
occupation_files = '/tmp/t-jizhao/debiaswe/data/professions.json'

In [13]:
def load_professions(occupation_files = occupation_files):
    with open(occupation_files, 'r') as f:
        professions = json.load(f)
    print('Loaded professions\n' +
          'Format:\n' +
          'word,\n' +
          'definitional female -1.0 -> definitional male 1.0\n' +
          'stereotypical female -1.0 -> stereotypical male 1.0')
    return professions

In [14]:
professions_raw = load_professions()
print(len(professions_raw))
print(professions_raw[10:30])

Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0
320
[['alter_ego', 0.0, 0.0], ['ambassador', 0.0, 0.7], ['analyst', 0.0, 0.4], ['anthropologist', 0.0, 0.4], ['archaeologist', 0.0, 0.6], ['archbishop', 0.4, 0.5], ['architect', 0.1, 0.6], ['artist', 0.0, -0.2], ['artiste', -0.1, -0.2], ['assassin', 0.1, 0.8], ['assistant_professor', 0.1, 0.4], ['associate_dean', 0.0, 0.4], ['associate_professor', 0.0, 0.4], ['astronaut', 0.1, 0.8], ['astronomer', 0.1, 0.5], ['athlete', 0.0, 0.7], ['athletic_director', 0.1, 0.7], ['attorney', 0.0, 0.3], ['author', 0.0, 0.1], ['baker', 0.0, -0.1]]


In [ ]:
show_bias(en_ori_E, en_ori_gd, 'ori EN')

In [ ]:
show_bias(en_ali_E, en_ali_gd, 'aligned EN')

In [ ]:
show_bias(en_mul_E, -en_mul_gd, 'multi EN')

In [ ]:
es_m_to_plot = [en2es[x][0] for x in m_to_plot if en2es[x] != []]
es_n_to_plot = ['enfermera', 'enfermero', 'casero', 'alguacil', 'presidente', 'coronel', 'cabeza', 'editorial', 'escucha' ]

In [ ]:
show_bias(es_ori_E, es_ori_gd, 'ori ES', es_m_to_plot, es_n_to_plot, \
          es_female_stereotyped_prof, es_male_stereotyped_prof, es_neutral_prof)

In [ ]:
show_bias(es_mul_E, es_mul_gd, 'mul ES', es_m_to_plot, es_n_to_plot, \
          es_female_stereotyped_prof, es_male_stereotyped_prof, es_neutral_prof)

In [ ]:
show_bias(es_ali_E, -es_ali_gd, 'aligned ES', es_m_to_plot, es_n_to_plot, \
          es_female_stereotyped_prof, es_male_stereotyped_prof, es_neutral_prof)

### build a classifier
- Train a classifier on EN gender def; evaluated on aligned ES

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [ ]:
def get_X_y(E, female_def, male_def):
    X_en_f = np.array([E.v(x) for x in female_def if x in E.words])
    X_en_m = np.array([E.v(x) for x in male_def if x in E.words])
    y_en = np.array([1] * len(X_en_f) + [0] * len(X_en_m))
    X_en = np.concatenate((X_en_f, X_en_m), axis = 0)
    print(len(X_en_f), len(X_en_m))
    return X_en, y_en

In [ ]:
X_en, y_en = get_X_y(en_ori_E, female_def, male_def)
X_es, y_es = get_X_y(es_ori_E, es_female_def, es_male_def)

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C':[0.1, 1, 10]}
svc = SVC(gamma='scale')
clf = GridSearchCV(svc, parameters, cv=5)
clf.fit(X_en, y_en) 

In [ ]:
clf.best_score_, clf.best_params_

In [ ]:
print(accuracy_score(clf.predict(X_en), y_en))
confusion_matrix(clf.predict(X_en), y_en)

In [ ]:
glove_file = '/home/t-jizhao/Github/fasttext/glove.vec.txt'
glove_en = we.WordEmbedding(glove_file)
X_en_g, y_en_g = get_X_y(glove_en, female_def, male_def)
clf = SVC(kernel = 'linear')
clf.fit(X_en_g, y_en_g) 
print(accuracy_score(clf.predict(X_en_g), y_en_g))
confusion_matrix(clf.predict(X_en_g), y_en_g)

In [ ]:
# on EN-biased
female_biased_en = [x for x in neutral_prof \
                    if np.dot(en_ori_E.v(x), en_ori_E.v('he')) \
                    < np.dot(en_ori_E.v(x), en_ori_E.v('she'))]
male_biased_en = [x for x in neutral_prof \
                    if np.dot(en_ori_E.v(x), en_ori_E.v('he')) \
                    > np.dot(en_ori_E.v(x), en_ori_E.v('she'))]
X_bias_en, y_bias_en = get_X_y(en_ori_E, female_biased_en, male_biased_en)

In [ ]:
print(clf.score(X_bias_en, y_bias_en))
confusion_matrix(clf.predict(X_bias_en), y_bias_en)

In [ ]:
np.average([np.dot(en_ori_E.v(x), en_ori_gd) \
            for x in female_biased_en]),\
np.average([np.dot(en_ori_E.v(x), en_ori_gd)\
            for x in male_biased_en])

In [ ]:
#directly on ES
clf.score(X_es, y_es)
confusion_matrix(clf.predict(X_es), y_es)

- Train classifier on ES gender def; evaluate on gender biased
    - using original ES embeddings

In [ ]:
clf_es = SVC(kernel = 'linear')
clf_es.fit(X_es, y_es) 
clf_es.score(X_es, y_es)

In [ ]:
X_neutral_es, y_neutral_es = get_X_y(es_ori_E, es_neutral_f, es_neutral_m)
print(X_neutral_es.shape, y_neutral_es.shape)

In [ ]:
print(clf_es.score(X_neutral_es, y_neutral_es))
confusion_matrix(clf_es.predict(X_neutral_es), y_neutral_es)

 - using multi ES embeddings (Orthogonal Procrustes)

In [ ]:
X_es_multi, y_es_multi = get_X_y(es_mul_E, es_female_def, es_male_def)
clf_es_mul = SVC(kernel = 'linear')
clf_es_mul.fit(X_es_multi, y_es_multi) 
print(clf_es_mul.score(X_es_multi, y_es_multi))

In [ ]:
X_neutral_es_mul, y_neutral_es_mul = get_X_y(es_mul_E, es_neutral_f, es_neutral_m)
print(clf_es_mul.score(X_neutral_es_mul, y_neutral_es_mul))
confusion_matrix(clf_es_mul.predict(X_neutral_es_mul), y_neutral_es_mul)

- using aligned ES embeddings

In [ ]:
X_es_ali, y_es_ali = get_X_y(es_ali_E, es_female_def, es_male_def)
clf_es_ali = SVC(kernel = 'linear')
clf_es_ali.fit(X_es_ali, y_es_ali) 
print(clf_es_ali.score(X_es_ali, y_es_ali))

In [ ]:
X_neutral_es_ali, y_neutral_es_ali = get_X_y(es_ali_E, es_neutral_f, es_neutral_m)
print(clf_es_ali.score(X_neutral_es_ali, y_neutral_es_ali))
confusion_matrix(clf_es_ali.predict(X_neutral_es_ali), y_neutral_es_ali)

In [ ]:
import random
random.seed(2)

In [ ]:
def plot_bias_proj(E, gender_direction, ll, \
                  es_neutral_m, es_neutral_f):
    female_stereotyped_gender = {x: np.dot(E.v(x), gender_direction) \
                                 for x in es_neutral_f if x in E.words}
    male_stereotyped_gender = {x: np.dot(E.v(x), gender_direction) \
                               for x in es_neutral_m if x in E.words}
    
    female_p = list(female_stereotyped_gender.keys())
    male_p = list(male_stereotyped_gender.keys())
    
    y_female = {female_p[x]:x/(len(female_p)) for x in range(len(female_p))}
    y_male = {male_p[x]:x/(len(male_p)) for x in range(len(male_p))}
    fig = plt.figure()
    fig.set_size_inches(8, 6)
    
    f, = plt.plot(list(female_stereotyped_gender.values()), \
                  list(y_female.values()), 'ro', label = 'Female Biased')
    m, = plt.plot(list(male_stereotyped_gender.values()), \
                  list(y_male.values()), 'ko', label = 'Male Biased')
    
    plt.yticks([])
    
    to_anno = random.sample(range(len(female_p)), 8)
    for x in to_anno:
        x_l = female_stereotyped_gender[female_p[x]] 
        y_l = y_female[female_p[x]]
        plt.annotate(female_p[x], (x_l, y_l ), fontsize=15)

    for x in to_anno:
        ann_x = male_stereotyped_gender[male_p[x]]
        plt.annotate(male_p[x], (ann_x, y_male[male_p[x]]), fontsize=15)
    
    plt.xticks(fontsize=17)
    plt.title(f"Biased proj in {ll}")
    plt.show()
    plt.close()

In [ ]:
plot_bias_proj(es_ori_E, es_ori_gd, 'ori ES', es_neutral_m, es_neutral_f)

In [ ]:
plot_bias_proj(es_ali_E, -es_ali_gd, 'aligned ES', es_neutral_m, es_neutral_f)

In [ ]:
# follow EN bias, what is the bias score for F-biased words and M-biased words
#F_biased + M_biased != len(es_single_lexo)
def cal_es_singleton_bias(E):
    O, F, M = [], [], []
    for f,m in es_gender_pairs:
        overall_bias = np.average([abs(dis(E, es_single_lexo[x], f) - \
                                       dis(E, es_single_lexo[x], m)) \
                                   for x in es_single_lexo])
        F_bias = np.average([abs(dis(E, es_single_lexo[x], f) - \
                                       dis(E, es_single_lexo[x], m)) \
                                   for x in en_biased_f])
        M_bias = np.average([abs(dis(E, es_single_lexo[x], f) - \
                                       dis(E, es_single_lexo[x], m)) \
                                   for x in en_biased_m])
        O.append(overall_bias)
        F.append(F_bias)
        M.append(M_bias)
    return np.average(O), np.average(F), np.average(M)

In [691]:
print("Bias analysis using single-format professions lexicons")
overall_bias, F_bias, M_bias = cal_bias(es_ori_E)
print(f"in original ES, average bias:{overall_bias}, avg F_bias:{F_bias}, avg M_bias:{M_bias}")
overall_bias, F_bias, M_bias = cal_bias(es_mul_E)
print(f"in sup-aligned ES, average bias:{overall_bias}, avg F_bias:{F_bias}, avg M_bias:{M_bias}")
overall_bias, F_bias, M_bias = cal_bias(es_ali_E)
print(f"in un-aligned ES, average bias:{overall_bias}, avg F_bias:{F_bias}, avg M_bias:{M_bias}")

Bias analysis using single-format professions lexicons
in original ES, average bias:0.03957142403605304, avg F_bias:0.03460703590125949, avg M_bias:0.0415010569749377
in sup-aligned ES, average bias:0.039571438976242365, avg F_bias:0.034607051940703834, avg M_bias:0.04150107266785084
in un-aligned ES, average bias:0.05585759617176103, avg F_bias:0.04485674078912578, avg M_bias:0.060115123595827616


- ##  Bias analysis using verbs

In [647]:
es_verbs = []
with open('fasttext/es_verbs.csv', 'r') as f:
    count = 0
    for line in f:
        tokens = line.strip().split(',')
        es_verbs.append(tokens[1].lower())

In [694]:
ori, ali = [], []
for f, m in es_gender_pairs:
    O_ori = np.average([abs(dis(es_ori_E, x, m) - \
                                       dis(es_ori_E, x, f)) \
                                   for x in es_verbs])
    O_ali = np.average([abs(dis(es_ali_E, x, m) - \
                                       dis(es_ali_E, x, f)) \
                                   for x in es_verbs])
    ori.append(O_ori), ali.append(O_ali)

In [695]:
print(f"Bias on ES verbs before aligment: {np.average(ori)}, after_alignment: {np.average(ali)}")

Bias on ES verbs before aligment: 0.02664801837893686, after_alignment: 0.026637466510928114


In [697]:
# json.dump(es_gender_pairs, open('fasttext/es_gender_pairs.json', 'w'))

In [ ]:
# female_def_professions_raw = [x for x in professions_raw if x[0] in female_def]
# male_def_professions_raw = [x for x in professions_raw if x[0] in male_def ]
# neutral_professions_raw = [x for x in professions_raw if (x not in female_def_professions_raw and
#                            x not in male_def_professions_raw)]
# female_def_prof = [x[0] for x in female_def_professions_raw \
#                            if x[0].lower() in vocab]
# male_def_prof = [x[0] for x in male_def_professions_raw \
#                          if x[0].lower() in vocab]
# neutral_prof = [x[0] for x in neutral_professions_raw \
#                 if x[0].lower() in vocab]
# print(len(female_def_prof))
# print(len(male_def_prof))
# print(len(neutral_prof))

- ## Bias with sentiment

In [69]:
pleasant = [x.strip()  for x in open('pleasant.txt', 'r').readlines()]
unpleasant = [x.strip()  for x in open('unpleasant.txt', 'r').readlines()]

In [70]:
es_pleasant = [en2es[x][0] for x in pleasant]
es_unpleasant = [en2es[x][0] for x in unpleasant if en2es[x] != []]
print(len(es_pleasant), len(es_unpleasant))

25 25


In [71]:
def get_bias_diff(ori_E, ali_E, words, gender_def_pairs):
    ori, ali = [], []
    for f, m in gender_def_pairs:
        O_ori = np.average([abs(dis(ori_E, x, m) - \
                                           dis(ori_E, x, f)) \
                                       for x in words])
        O_ali = np.average([abs(dis(ali_E, x, m) - \
                                           dis(ali_E, x, f)) \
                                       for x in words])
        ori.append(O_ori), ali.append(O_ali)
    print(f"Bias on words before aligment: {np.average(ori)}, after_alignment: {np.average(ali)}")

- In EN

In [798]:
get_bias_diff(en_ori_E, en_ali_E, pleasant, gender_pairs)
get_bias_diff(en_ori_E, en_ali_E, unpleasant, gender_pairs)

Bias on words before aligment: 0.04284609885513783, after_alignment: 0.04284764948487282
Bias on words before aligment: 0.03575442790985107, after_alignment: 0.03575175215303898


In [72]:
get_bias_diff(en_sup_E, en_sup_E, pleasant, gender_pairs)
get_bias_diff(en_sup_E, en_sup_E, unpleasant, gender_pairs)

Bias on words before aligment: 0.03740838485956192, after_alignment: 0.03740838485956192
Bias on words before aligment: 0.03146131804585457, after_alignment: 0.03146131804585457


- In ES

In [72]:
get_bias_diff(es_ori_E, es_sup_E, es_pleasant, es_gender_pairs)
get_bias_diff(es_ori_E, es_sup_E, es_unpleasant, es_gender_pairs)

Bias on words before aligment: 0.02887786931461758, after_alignment: 0.02887786931461758


In [73]:
get_bias_diff(es_sup_E, es_sup_E, es_pleasant, es_gender_pairs)
get_bias_diff(es_sup_E, es_sup_E, es_unpleasant, es_gender_pairs)

Bias on words before aligment: 0.033097195294168255, after_alignment: 0.033097195294168255
Bias on words before aligment: 0.028620654675695634, after_alignment: 0.028620654675695634


In [35]:
# en_neutral_score = {x[0]:[x[1], x[2]] for x in neutral_professions_raw}
# en_biased_m = [x for x in en_neutral_score if float(en_neutral_score[x][1]) > 0]
# en_biased_f = [x for x in en_neutral_score if float(en_neutral_score[x][1]) < -0]

In [72]:
# O, F, M = cal_bias_paired(en_ori_E, neutral_prof, neutral_prof, neutral_prof, gender_pairs)
# print(f"in original EN, average bias:{O}, avg F_bias:{F}, avg M_bias:{M}")

in original EN, average bias:0.04776176616868556, avg F_bias:0.7566125864966737, avg M_bias:0.7455528793277875


In [73]:
# O, F, M = cal_bias_paired(en_sup_fr_E, neutral_prof, neutral_prof, neutral_prof, gender_pairs)
# print(f"In algined EN, average bias:{O}, avg F_bias:{F}, avg M_bias:{M}")

In algined EN, average bias:0.040779857862464804, avg F_bias:0.6958112337812782, avg M_bias:0.6892272675747948


In [76]:
# O, F, M = cal_bias_paired(en_sup_zh_E, neutral_prof, neutral_prof, neutral_prof, gender_pairs)
# print(f"In algined EN, average bias:{O}, avg F_bias:{F}, avg M_bias:{M}")

In algined EN, average bias:0.0503315428418918, avg F_bias:0.7809175156299197, avg M_bias:0.7663548305408632


In [45]:
# es_female_stereotyped_prof = [en2es[x][0] for x in female_def_prof if en2es[x] != []]
# es_male_stereotyped_prof = [en2es[x][0] for x in male_def_prof if en2es[x] != []]
# es_neutral_prof = [en2es[x][0] for x in neutral_prof if en2es[x] != []]
# print(len(es_female_stereotyped_prof), len(es_male_stereotyped_prof), len(es_neutral_prof))

6 16 242


In [46]:
# es_female_def, es_male_def = [], []
# for x in female_def:
#     if x in en2es:
#         if len(en2es[x]) > 1:
#             for t in en2es[x]:
#                 if t[-1] == 'a' or t[-2:] == 'as':
#                     es_female_def.append(t)
#                     break
#         else:
#             es_female_def.append(x)
# for x in male_def:
#     if x in en2es:
#         if len(en2es[x]) > 1:
#             for t in en2es[x]:
#                 if t[-1] == 'o' or t[-2:] == 'os':
#                     es_male_def.append(t)
#                     break 
#         else:
#             es_male_def.append(x)

In [120]:
# es_female_def[:3], es_male_def[:3]

In [47]:
# es_neutral_profs = [en2es[x] for x in neutral_prof if len(en2es[x]) > 1]
# es_neutral_profs_en = [x for x in neutral_prof if len(en2es[x]) > 1]
# print(len(es_neutral_profs), len(es_neutral_profs_en))

126 126


In [48]:
# es_neutral_f, es_neutral_m = [], []
# es_neutral_en = []
# ignored = []
# for x in range(len(es_neutral_profs)):
#     flag = 0
#     tmp = ['', '']
#     for t in es_neutral_profs[x]:
#         if t.endswith('a'):
#             tmp[0] = t
#         else:
#             tmp[1] = t
#         if tmp[0] != '' and tmp[1] != '':
#             es_neutral_f.append(tmp[0])
#             es_neutral_m.append(tmp[1])
#             flag = 1
#             es_neutral_en.append(es_neutral_profs_en[x])
#             break
#     if flag == 0:
#         ignored.append(es_neutral_profs[x])
# print(len(es_neutral_f), len(es_neutral_m), len(es_neutral_en))

87 87 87


#### cos_sim between occ and gender word (feminine prof, 'ella') - (masculine prof, 'el')

In [50]:
def show_bias_proj(E, gd, es_neutral_m, es_neutral_f):
    female_biased_es = [es_neutral_f[x] for x in range(len(es_neutral_f)) \
                        if np.dot(E.v(es_neutral_m[x]), E.v('macho')) \
                        < np.dot(E.v(es_neutral_f[x]), E.v('femenino'))]
    male_biased_es = [es_neutral_m[x] for x in range(len(es_neutral_m)) \
                        if np.dot(E.v(es_neutral_m[x]), E.v('macho')) \
                        > np.dot(E.v(es_neutral_f[x]), E.v('femenino'))]
    print(f"len(male/female_biased_es prof): {len(male_biased_es), len(female_biased_es)}")
    print(male_biased_es[:3], female_biased_es[:3])
    print("male-biased occupations projected on gender direction:", \
          np.average([np.dot(E.v(x), gd) for x in male_biased_es]))
    print("female-biased occupations projected on gender direction:", \
          np.average([np.dot(E.v(x), gd) for x in female_biased_es]))
    return female_biased_es, male_biased_es

In [51]:
female_biased_es, male_biased_es = show_bias_proj(es_ori_E, es_ori_gd, es_neutral_m, es_neutral_f)

len(male/female_biased_es prof): (23, 64)
['aventurero', 'asesino', 'locutor'] ['administradora', 'embajadora', 'analista']
male-biased occupations projected on gender direction: 0.08865328
female-biased occupations projected on gender direction: -0.2283584


In [73]:
female_biased_es_ali, male_biased_es_ali = \
show_bias_proj(es_ali_E, es_ali_gd, es_neutral_m, es_neutral_f)

len(male/female_biased_es prof): (19, 68)
['asesino', 'locutor', 'carnicero'] ['administradora', 'aventurera', 'embajadora']
male-biased occupations projected on gender direction: -0.019205257
female-biased occupations projected on gender direction: 0.2831623


In [95]:
_, _ = show_bias_proj(es_sup_E, es_sup_gd, es_neutral_m, es_neutral_f)

len(male/female_biased_es prof): (26, 61)
['aventurero', 'asesino', 'locutor'] ['administradora', 'embajadora', 'analista']
male-biased occupations projected on gender direction: 0.035033684
female-biased occupations projected on gender direction: 0.27913365


In [352]:
set(female_biased_es) - set(female_biased_es_ali), \
set(female_biased_es_ali) - set(female_biased_es)

({'caricaturista', 'ministra', 'policía'},
 {'aventurera',
  'dentista',
  'diplomática',
  'enviada',
  'mecánica',
  'peluquera',
  'secretaría'})

- #### 2.1 Single format for masculine and feminine occupations

In [52]:
# es_single_lexo = {x:en2es[x][0] for x in neutral_prof if len(en2es[x]) == 1}
# es_single_lexo
# len(es_single_lexo), es_single_lexo['alderman']
# print(len(en_biased_f), len(en_biased_m))
# en_neutral_score['caretaker']

### Assumption:
- Assume the original points in the gender direction means gender neutral 
- Assume all these occupations should have same distance to male/female
- Diff(dis_to_he, dis_to_she)


- #### 2.2 Bias analysis with two-format occupations

In [56]:
# es_gender_pairs = [['mujer', 'hombre'], ['chica', 'chico'], ['madre', 'padre'], ['hija', 'hijo'],
#                   ['femenino', 'masculino'], ['su', 'su'], ['maría', 'juan'], ['hembra', 'macho'],
#                   ['ella', 'él']]

In [58]:
#for paired occupations in ES
o, F, M = cal_bias_paired(es_ori_E, es_neutral_m, es_neutral_f, es_neutral_en, es_gender_pairs)
print(f"in original ES, average bias:{o}, avg F_bias:{F}, avg M_bias:{M}")
o, F, M = cal_bias_paired(es_ali_E, es_neutral_m, es_neutral_f, es_neutral_en, es_gender_pairs)
print(f"in en-aligned ES, average bias:{o}, avg F_bias:{F}, avg M_bias:{M}")

in original ES, average bias:0.07259895407748176, avg F_bias:0.7321097015535237, avg M_bias:0.7776997056819432


In [50]:
#for both singleton + paired occs in ES:
xm = es_neutral_m #+ [es_single_lexo[x] for x in es_single_lexo]
xf = es_neutral_f #+ [es_single_lexo[x] for x in es_single_lexo]
es_en = es_neutral_en #+ [x for x in es_single_lexo]

## Bias in DE

In [79]:
en2de = defaultdict(list)
with open('/tmp/t-jizhao/data/fasttext/en-de.txt', 'r') as f:
    for line in f:
        en, de = line.strip().split()
        en2de[en].append(de)

In [52]:
# m_ind = "-er, -el, -ling, -ich, -ig, -ner, -ismus, -or, -us, -eich, -ant"
# f_ind = "-e, -ie, -heit, -ei, -in, -ik, -keit, – schaft, -ung, -tät, -ur, -tion"
# n_ind = "-chen, -o, -lein, -en, -il, -ma, -tel, -ment, -nis, -tum, -um"

In [53]:
# de_mks, de_fks, de_nks = [], [], []
# for mi in m_ind.strip().split(', '):
#     de_mks.append(mi[1:])

# for fi in f_ind.strip().split(', '):
#     de_fks.append(fi[1:])
    
# for ni in n_ind.strip().split(', '):
#     de_nks.append(ni[1:])

In [54]:
# de_profs = {}
# de_single_profs = {}
# for x in neutral_prof:
    
#     if len(en2de[x]) == 1:
#         de_single_profs[x] = en2de[x][0]
#         continue
#     tmp = ['', '', '']
#     for t in en2de[x]:
#         for i in de_mks:
#             if t.endswith(i):
#                 tmp[0] = t
#                 break
#         for i in de_fks:
#             if t.endswith(i):
#                 tmp[1] = t
#                 break
#         for i in de_nks:
#             if t.endswith(i):
#                 tmp[2] = t
#                 break
#         if tmp[0] != '' and tmp[1] != '':
#             de_profs[x] = tmp[:]
#             break


In [58]:
# de_prof_m = [de_profs[x][0] for x in de_profs] + [de_single_profs[x] for x in de_single_profs]
# de_prof_f = [de_profs[x][1] for x in de_profs] + [de_single_profs[x] for x in de_single_profs]
# de_prof_en = [x for x in de_profs] + [x for x in de_single_profs]

In [59]:
# de_gender_def_pairs = []
# for f,m in gender_pairs:
#     if en2de[f.lower()] != [] and en2de[m.lower()] != []:
#         de_gender_def_pairs.append([en2de[f.lower()][0], en2de[m.lower()][0]])

In [90]:
de_pleasant = [en2de[x][0] for x in pleasant if en2de[x] != []]
de_unpleasant = [en2de[x][0] for x in unpleasant if en2de[x] != []]
print(len(de_pleasant), len(de_unpleasant))

25 25


In [225]:
print("Bias in sentiment words")
print("---pleasant words:")
get_bias_diff(de_ori_E, de_ali_E, de_pleasant, de_gender_pairs)
print("---unpleasant words:")
get_bias_diff(de_ori_E, de_ali_E, de_unpleasant, de_gender_pairs)

Bias in sentiment words
---pleasant words:
Bias on words before aligment: 0.047716377211941614, after_alignment: 0.05466125662955973
---unpleasant words:
Bias on words before aligment: 0.04536173863543405, after_alignment: 0.049679737471871904


In [91]:
print("Bias in sentiment words")
print("---pleasant words:")
get_bias_diff(de_sup_E, de_sup_E, de_pleasant, de_gender_pairs)
print("---unpleasant words:")
get_bias_diff(de_sup_E, de_sup_E, de_unpleasant, de_gender_pairs)

Bias in sentiment words
---pleasant words:
Bias on words before aligment: 0.04487085190084246, after_alignment: 0.04487085190084246
---unpleasant words:
Bias on words before aligment: 0.0416201490826077, after_alignment: 0.0416201490826077


In [854]:
# width = 15
# with open('fasttext/pleasants.txt', 'w') as f:
#     f.write('{: <{}}'.format('EN', width) + '{: <{}}'.format('ES', width) + \
#             '{: <{}}'.format('DE', width) +  '{: <{}}'.format('FR', width)+ '\n')
#     for idx in range(len(pleasant)):
#         f.write('{: <{}}'.format(pleasant[idx], width) + '{: <{}}'.format(es_pleasant[idx], width) + \
#             '{: <{}}'.format(de_pleasant[idx], width) + '{: <{}}'.format(fr_pleasant[idx], width) + '\n')

## Bias in FR

In [61]:
en2fr = defaultdict(list)
with open('/home/t-jizhao/Github/data/fasttext/en-fr.txt', 'r') as f:
    for line in f:
        en, fr = line.strip().split()
        en2fr[en].append(fr)

In [96]:
m_ind = "-age, -ment, -il, -ail, -eil, -euil, -é, -eau, -eu, -er, -oir, -isme, -ing, -ard, -am, -um, -em, -it, -est, -an, -and, -ant, -ent, -in, -int, -om, -ond, -ont, -ème, -ège"
f_ind = "-tion, -sion, -son, -ure, -ude, -ade, -ée, -té, -ière, -euse, -ance, -ence, -ine, -elle, -esse, -ette"
mks, fks, nks = [], [], []
for mi in m_ind.strip().split(', '):
    mks.append(mi[1:])

for fi in f_ind.strip().split(', '):
    fks.append(fi[1:])

In [116]:
fr_profs = {}
fr_single_profs = {}
for x in neutral_prof:
    if len(en2fr[x])  == 1:
        fr_single_profs[x] = en2fr[x][0]
        continue
    tmp = ['', '', '']
    for t in en2fr[x]:
        for i in mks:
            if t.endswith(i):
                tmp[0] = t
                break
        for i in fks:
            if t.endswith(i):
                tmp[1] = t
                break
        if tmp[0] != '' and tmp[1] != '':
            fr_profs[x] = tmp[:2]
            break

In [117]:
len(fr_single_profs), len(fr_profs)

(117, 10)

In [39]:
import re

In [40]:
rx = re.compile('([\'\[,\]])')

In [50]:
fr_profs = {}
with open('fr2plot.txt', 'r') as f:
    for line in f:
        line = rx.sub('', line.strip())
        tokens = line.strip().split()
        if len(tokens) == 2:
            tokens.append(tokens[-1])
        fr_profs[tokens[0]]= tokens[1:]

In [87]:
# fr_gender_def_pairs = []
# for f,m in gender_pairs:
#     if en2fr[f.lower()] != [] and en2fr[m.lower()] != []:
#         fr_gender_def_pairs.append([en2fr[f.lower()][0], en2fr[m.lower()][0]])

In [91]:
# json.dump(fr_gender_pairs, open('fasttext/fr_gender_pairs.json', 'w'))

In [851]:
fr_pleasant = [en2fr[x][0] for x in pleasant if en2fr[x] != []]
fr_unpleasant = [en2fr[x][0] for x in unpleasant if en2fr[x] != []]
print(len(fr_pleasant), len(fr_unpleasant))

25 25


In [849]:
[x for x in pleasant if en2fr[x] == []]

['vacation']

In [852]:
print("Bias in sentiment words")
print("---pleasant words:")
get_bias_diff(fr_ori_E, fr_ali_E, fr_pleasant, fr_gender_pairs)
print("---unpleasant words:")
get_bias_diff(fr_ori_E, fr_ali_E, fr_unpleasant, fr_gender_pairs)

Bias in sentiment words
---pleasant words:
Bias on words before aligment: 0.052481612940318875, after_alignment: 0.0626819814601913
---unpleasant words:
Bias on words before aligment: 0.059130257647484544, after_alignment: 0.06513589754293206


In [132]:
fr_prof_m = [fr_profs[x][0] for x in fr_profs] + [fr_single_profs[x] for x in fr_single_profs]
fr_prof_f = [fr_profs[x][1] for x in fr_profs] + [fr_single_profs[x] for x in fr_single_profs]
fr_prof_en = [x for x in fr_profs] + [x for x in fr_single_profs]